In [1]:
import pandas as pd

In [2]:
df_0 = pd.read_csv('ships_1.csv')
df_0

,name,lat,lng
0,Black Pearl,20.664865,-80.709747
1,Cobra,20.664868,-80.709740
2,Flying Dutchman,20.664878,-80.709941
3,Empress,NaN,NaN
4,,20.664875,-80.709777


In [3]:
df_0[df_0.isnull().any(axis=1)]

,name,lat,lng
3,Empress,NaN,NaN


In [4]:
df_0.iloc[-1]['name']

' '

In [5]:
df_0['name']=df_0['name'].str.strip()
df_0.iloc[-1]['name']

''

In [6]:
df_0[df_0.isnull().any(axis=1)]

,name,lat,lng
3,Empress,NaN,NaN


In [7]:
import numpy as np

In [11]:
mask = df_0['name'].str.strip() ==''
#mask = [False,False,False,False,True]
df_0.loc[mask,'name']=np.nan
df_0

,name,lat,lng
0,Black Pearl,20.664865,-80.709747
1,Cobra,20.664868,-80.709740
2,Flying Dutchman,20.664878,-80.709941
3,Empress,NaN,NaN
4,NaN,20.664875,-80.709777


In [12]:
df_0[df_0.isnull().any(axis=1)]

,name,lat,lng
3,Empress,NaN,NaN
4,NaN,20.664875,-80.709777


In [14]:
df_1 = pd.read_csv('ships_2.csv')
df_1

,name,lat,lng
0,Black Pearl,20.664865,-80.709747
1,Cobra,20.664868,-80.709740
2,Flying Dutchman,20.664878,-80.709941
3,Empress,NaN,NaN


In [16]:
import pandera as pa

In [17]:
schema = pa.DataFrameSchema({
    'name': pa.Column(pa.String),
    'lat': pa.Column(
        pa.Float,
        nullable=True,
        checks=pa.Check(
            lambda v: v >= -90 and v <= 90,
            element_wise=True,
        ),
    ),
    'lng': pa.Column(
        pa.Float,
        nullable=True,
        checks=pa.Check(
            lambda v: v >= -180 and v <= 180,
            element_wise=True,
        ),
    ),
})

In [18]:
schema.validate(df_1)

,name,lat,lng
0,Black Pearl,20.664865,-80.709747
1,Cobra,20.664868,-80.709740
2,Flying Dutchman,20.664878,-80.709941
3,Empress,NaN,NaN


In [20]:
df_2 = pd.read_csv('heights_0.csv')
df_2

,name,grade,height
0,Adam,1,31.7
1,Beth,1,74.9
2,Chris,12,72.3
3,Dana,12,61.8


In [30]:
max_heights = pd.DataFrame([[1, 32],], columns=['grade', 'max_height'])
max_heights

,grade,max_height
0,1,32


In [28]:
mdf = pd.merge(df_2, max_heights, how='left')
mdf

,name,grade,height,max_height
0,Adam,1,31.7,32.0
1,Beth,1,74.9,32.0
2,Chris,12,72.3,NaN
3,Dana,12,61.8,NaN


In [31]:
df_2[mdf['height'] > mdf['max_height']]

,name,grade,height
1,Beth,1,74.9


In [32]:
df_3 = pd.read_csv('rides_0.csv')
df_3

,name,plate,distance
0,Gomez,1XZ2,3.70
1,Morticia,NaN,2.10
2,Fester,,3.40
3,Lurch,Q38X3,-3.20
4,NaN,03A,14.30
5,Wednesday,A,0.30
6,Pugsley,ZF003,153.14


In [33]:
null_mask = df_3.isnull().any(axis=1)
null_mask

0    False
1     True
2    False
3    False
4     True
5    False
6    False
dtype: bool

In [34]:
df_3[null_mask]

,name,plate,distance
1,Morticia,NaN,2.1
4,NaN,03A,14.3


In [36]:
plate_mask = ~df_3['plate'].str.match(r'^[0-9A-Z]{3,}', na=False)
plate_mask

0    False
1     True
2     True
3    False
4    False
5     True
6    False
Name: plate, dtype: bool

In [37]:
df_3[plate_mask]

,name,plate,distance
1,Morticia,NaN,2.1
2,Fester,,3.4
5,Wednesday,A,0.3


In [40]:
dist_mask = df_3['distance'] < 0
dist_mask

0    False
1    False
2    False
3     True
4    False
5    False
6    False
Name: distance, dtype: bool

In [41]:
df_3[dist_mask]

,name,plate,distance
3,Lurch,Q38X3,-3.2


In [42]:
mask = null_mask|plate_mask|dist_mask
mask

0    False
1     True
2     True
3     True
4     True
5     True
6    False
dtype: bool

In [43]:
df_3[mask]

,name,plate,distance
1,Morticia,NaN,2.1
2,Fester,,3.4
3,Lurch,Q38X3,-3.2
4,NaN,03A,14.3
5,Wednesday,A,0.3
